In [1]:
import pandas as pd
import cassiopeia as cass

In [2]:
cass.set_riot_api_key("AN API KEY")

In [3]:
def get_stats(player, new_df):
    
    first_blood = player.stats.first_blood_kill
    double_kill = player.stats.double_kills
    triple_kill = player.stats.triple_kills
    quadra_kills = player.stats.quadra_kills
    penta_kills = player.stats.penta_kills
    kda = player.stats.kda
    kills = player.stats.kills
    deaths = player.stats.deaths
    assists = player.stats.assists
    CS = player.stats.total_minions_killed + player.stats.neutral_minions_killed
    damage_dealt = player.stats.total_damage_dealt_to_champions
    damage_taken = player.stats.total_damage_taken
    gold_earned = player.stats.gold_earned
    turret_damage = player.stats.damage_dealt_to_turrets
    turret_kills = player.stats.turret_kills
    turret_takedowns = player.stats.turret_takedowns
    vision_wards_bought = player.stats.vision_wards_bought 
    vision_wards_placed = player.stats.vision_wards_placed 
    wards_placed = player.stats.wards_placed 
    wards_killed = player.stats.wards_killed 
    cc_score = player.stats.time_CCing_others



    new_df['First_Bloods'] += first_blood
    new_df['Double_Kills'] += double_kill
    new_df['Triple_Kills'] += triple_kill 
    new_df['Quadra_Kills'] += quadra_kills 
    new_df['Penta_Kills'] += penta_kills 
    new_df['KDA'] += kda 
    new_df['Kills'] += kills
    new_df['Deaths'] += deaths 
    new_df['Assists'] += assists 
    new_df['CS'] += CS 
    new_df['Damage_Dealt'] += damage_dealt 
    new_df['Damage_Taken'] += damage_taken 
    new_df['Gold_Earned'] += gold_earned 
    new_df['Turret_Damage'] += turret_damage 
    new_df['Turret_Kills'] += turret_kills 
    new_df['Turrt_Takedowns'] += turret_takedowns
    new_df['VisionW_Bought'] += vision_wards_bought 
    new_df['VisionW_Placed'] += vision_wards_placed 
    new_df['Wards_Placed'] += wards_placed 
    new_df['Wards_Killed'] += wards_killed 
    new_df['CC_Score'] += cc_score 
        

In [4]:
# functionizd version to get all the stats

# these parameters will be input from the website

# and data will be collected for the 5 players

def Lobby(side, rank, top_sum, top_champ, jung_sum, jung_champ, mid_sum, mid_champ, bot_sum, bot_champ, sup_sum, sup_champ, e_top, e_jung, e_mid, e_bot, e_sup):

    player1 = cass.Summoner(name=top_sum, region="NA")
    player2 = cass.Summoner(name=jung_sum, region="NA")
    player3 = cass.Summoner(name=mid_sum, region="NA")
    player4 = cass.Summoner(name=bot_sum, region="NA")
    player5 = cass.Summoner(name=sup_sum, region="NA")

    puuid_one = player1.puuid
    puuid_two = player2.puuid
    puuid_three = player3.puuid
    puuid_four = player4.puuid
    puuid_five = player5.puuid

    top_history = cass.MatchHistory(puuid=puuid_one,continent="AMERICAS", region='NA',begin_index = 0, end_index = 20,queue=cass.Queue.ranked_solo_fives)
    jg_history = cass.MatchHistory(puuid=puuid_two,continent="AMERICAS", region='NA',begin_index = 0, end_index = 20,queue=cass.Queue.ranked_solo_fives)
    mid_history = cass.MatchHistory(puuid=puuid_three,continent="AMERICAS", region='NA',begin_index = 0, end_index = 20,queue=cass.Queue.ranked_solo_fives)
    bot_history = cass.MatchHistory(puuid=puuid_four,continent="AMERICAS", region='NA',begin_index = 0, end_index = 20,queue=cass.Queue.ranked_solo_fives)
    sup_history = cass.MatchHistory(puuid=puuid_five,continent="AMERICAS", region='NA',begin_index = 0, end_index = 20,queue=cass.Queue.ranked_solo_fives)

    new_df = pd.DataFrame()
    new_df['First_Bloods'] = [0]
    new_df['Double_Kills'] = [0]
    new_df['Triple_Kills'] = [0] 
    new_df['Quadra_Kills'] = [0] 
    new_df['Penta_Kills'] = [0] 
    new_df['KDA']= [0] 
    new_df['Kills']= [0]
    new_df['Deaths']= [0] 
    new_df['Assists']= [0] 
    new_df['CS']= [0] 
    new_df['Damage_Dealt'] = [0] 
    new_df['Damage_Taken'] = [0] 
    new_df['Gold_Earned']= [0] 
    new_df['Turret_Damage']= [0] 
    new_df['Turret_Kills']= [0] 
    new_df['Turrt_Takedowns']= [0]
    new_df['VisionW_Bought']= [0] 
    new_df['VisionW_Placed']= [0] 
    new_df['Wards_Placed']= [0] 
    new_df['Wards_Killed']= [0] 
    new_df['CC_Score']= [0] 
    

    stat_import = pd.DataFrame()
    stat_import['Side'] = [0]
    stat_import['Rank'] = [0]
    stat_import['Top_Current_Champ'] = [0]
    stat_import['Top_games_on_role'] = [0]
    stat_import['Top_games_on_champ'] = [0]
    stat_import['Top_Win'] = [0]
    stat_import['Top_Lose'] = [0]
    stat_import['Top_Win_2'] = [0]
    stat_import['Top_Win_5'] = [0]
    stat_import['Jg_Current_Champ'] = [0]
    stat_import['Jg_games_on_role'] = [0]
    stat_import['Jg_games_on_champ'] = [0]
    stat_import['Jg_Win'] = [0]
    stat_import['Jg_Lose'] = [0]
    stat_import['Jg_Win_2'] = [0]
    stat_import['Jg_Win_5'] = [0]
    stat_import['Mid_Current_Champ'] = [0]  
    stat_import['Mid_games_on_role'] = [0]
    stat_import['Mid_games_on_champ'] = [0]
    stat_import['Mid_Win'] = [0]
    stat_import['Mid_Lose'] = [0]
    stat_import['Mid_Win_2'] = [0]
    stat_import['Mid_Win_5'] = [0]
    stat_import['Bot_Current_Champ'] = [0]
    stat_import['Bot_games_on_role'] = [0]
    stat_import['Bot_games_on_champ'] = [0]
    stat_import['Bot_Win'] = [0]
    stat_import['Bot_Lose'] = [0]
    stat_import['Bot_Win_2'] = [0]
    stat_import['Bot_Win_5'] = [0]
    stat_import['Sup_Current_Champ'] = [0]
    stat_import['Sup_games_on_role'] = [0]
    stat_import['Sup_games_on_champ'] = [0]
    stat_import['Sup_Win'] = [0]
    stat_import['Sup_Lose'] = [0]
    stat_import['Sup_Win_2'] = [0]
    stat_import['Sup_Win_5'] = [0]


    stat_import['Side'] = [side]
    stat_import['Rank'] = [rank]
    stat_import['Top_Current_Champ'] = top_champ
    stat_import['Jg_Current_Champ'] = jung_champ
    stat_import['Mid_Current_Champ'] = mid_champ  
    stat_import['Bot_Current_Champ'] = bot_champ
    stat_import['Sup_Current_Champ'] = sup_champ
    stat_import['Top_Enemy_Champ'] = e_top
    stat_import['Jg_Enemy_Champ'] = e_jung
    stat_import['Mid_Enemy_Champ'] = e_mid
    stat_import['Bot_Enemy_Champ'] = e_bot
    stat_import['Sup_Enemy_Champ'] = e_sup

    try:
        match_count = 0
        for matches in top_history:
            for participants in matches.participants:
                if participants.summoner.puuid == puuid_one:    
                    if participants.individual_position.name == 'top_lane':
                        stat_import['Top_games_on_role'] += 1
                    if participants.champion.name == stat_import.iloc[0]['Top_Current_Champ']:
                        stat_import['Top_games_on_champ'] += 1
                    if participants.stats.win.real == 1:
                        stat_import['Top_Win'] += 1
                    elif participants.stats.win.real == 0:
                        stat_import['Top_Lose'] += 1
                    if match_count < 2:
                        if participants.stats.win.real == 1:
                            stat_import['Top_Win_2'] += 1  
                    if match_count < 5:
                        if participants.stats.win.real == 1:
                            stat_import['Top_Win_5'] += 1
                    match_count += 1
                    get_stats(participants,new_df)
                    break
    except:
        pass
    new_df.iloc[0:,5:] = new_df.iloc[0:,5:] / 20
    for column in new_df.columns:
        stat_import['Top_'+column] = new_df[column] 
        new_df[column] = [0] 
    try:
        match_count = 0
        for matches in jg_history:
            for participants in matches.participants:
                if participants.summoner.puuid == puuid_two:
                    if participants.individual_position.name == 'jungle':
                        stat_import['Jg_games_on_role'] += 1
                    if participants.champion.name == stat_import.iloc[0]['Jg_Current_Champ']:
                        stat_import['Jg_games_on_champ'] += 1
                    if participants.stats.win.real == 1:
                        stat_import['Jg_Win'] += 1
                    elif participants.stats.win.real == 0:
                        stat_import['Jg_Lose'] += 1
                    if match_count < 2:
                        if participants.stats.win.real == 1:
                            stat_import['Jg_Win_2'] += 1
                    if match_count < 5:
                        if participants.stats.win.real == 1:
                            stat_import['Jg_Win_5'] += 1
                    match_count += 1
                    get_stats(participants,new_df)
                    break
    except:
        pass
    new_df.iloc[0:,5:] = new_df.iloc[0:,5:] / 20
    for column in new_df.columns:
        stat_import['Jg_'+column] = new_df[column]
        new_df[column] = [0]
    try:
        match_count = 0
        for matches in mid_history:
            for participants in matches.participants:
                if participants.summoner.puuid == puuid_three:
                    if participants.individual_position.name == 'mid_lane':
                        stat_import['Mid_games_on_role'] += 1
                    if participants.champion.name == stat_import.iloc[0]['Mid_Current_Champ']:
                        stat_import['Mid_games_on_champ'] += 1
                    if participants.stats.win.real == 1:
                        stat_import['Mid_Win'] += 1
                    elif participants.stats.win.real == 0:
                        stat_import['Mid_Lose'] += 1
                    if match_count < 2:
                        if participants.stats.win.real == 1:
                            stat_import['Mid_Win_2'] += 1
                    if match_count < 5:
                        if participants.stats.win.real == 1:
                            stat_import['Mid_Win_5'] += 1
                    match_count += 1
                    get_stats(participants,new_df)
                    break
    except:
        pass
        new_df.iloc[0:,5:] = new_df.iloc[0:,5:] / 20
    for column in new_df.columns:
        stat_import['Mid_'+column] = new_df[column]
        new_df[column] = [0] 
    try:
        match_count = 0
        for matches in bot_history:
            for participants in matches.participants:
                if participants.summoner.puuid == puuid_four:
                    if participants.individual_position.name == 'bot_lane':
                        stat_import['Bot_games_on_role'] += 1
                    if participants.champion.name == stat_import.iloc[0]['Bot_Current_Champ']:
                        stat_import['Bot_games_on_champ'] += 1
                    if participants.stats.win.real == 1:
                        stat_import['Bot_Win'] += 1
                    elif participants.stats.win.real == 0:
                        stat_import['Bot_Lose'] += 1
                    if match_count < 2:
                        if participants.stats.win.real == 1:
                            stat_import['Bot_Win_2'] += 1
                    if match_count < 5:
                        if participants.stats.win.real == 1:
                            stat_import['Bot_Win_5'] += 1
                    match_count += 1
                    get_stats(participants,new_df)
                    break
    except:
        pass
    new_df.iloc[0:,5:] = new_df.iloc[0:,5:] / 20
    for column in new_df.columns:
        stat_import['Bot_'+column] = new_df[column] 
        new_df[column] = [0] 
    try:
        match_count = 0
        for matches in sup_history:
            for participants in matches.participants:
                if participants.summoner.puuid == puuid_five:
                    if participants.individual_position.name == 'utility':
                        stat_import['Sup_games_on_role'] += 1
                    if participants.champion.name == stat_import.iloc[0]['Sup_Current_Champ']:
                        stat_import['Sup_games_on_champ'] += 1
                    if participants.stats.win.real == 1:
                        stat_import['Sup_Win'] += 1
                    elif participants.stats.win.real == 0:
                        stat_import['Sup_Lose'] += 1
                    if match_count < 2:
                        if participants.stats.win.real == 1:
                            stat_import['Sup_Win_2'] += 1
                    if match_count < 5:
                        if participants.stats.win.real == 1:
                            stat_import['Sup_Win_5'] += 1
                    match_count += 1
                    get_stats(participants,new_df)
                    break
    except:
        pass
    new_df.iloc[0:,5:] = new_df.iloc[0:,5:] / 20
    for column in new_df.columns:
        stat_import['Sup_'+column] = new_df[column] 
        new_df[column] = [0]  


    stat_import.to_csv("lobby.csv",index=False,mode='w',header=True)

In [5]:
Lobby('Blue','Diamond','Artezzaa','Irelia','Prove','Graves','LCS MIDKING','Ahri','Quartze','Jinx','sketchybob','Lulu','Jayce','Hecarim','Sylas','Jinx','Leona')

Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/Artezzaa
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/Prove
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/LCS%20MIDKING
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/Quartze
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/sketchybob
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/TbAmHuYgAZxeaiwLN-ScDFTKuksDimRmnSgLe4EsW_3CS-0HUCIE5LqTwZk-Mp2lhQZ7SPsDMUIgXw/ids?start=0&count=20&queue=420
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4294488498
Making call: https://cdn.merakianalytics.com/riot/lol/resources/patches.json
Making call: https://ddragon.leagueoflegends.com/api/versions.json
Making call: https://ddragon.leagueoflegends.com/cdn/12.8.1/data/en_US/championFull.json
Making call: https://ddragon.leagueoflegends.com/realms/na.jso

C:\Users\Tigran\AppData\Local\Temp\ipykernel_22868\989418763.py:180: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  stat_import['Mid_'+column] = new_df[column]


Making call: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/i45rXAEwGxXCDnRRiCBSw_QxUUZHO0qpEHY6numx5vongs-3l38YURgVWGUznM4XW8h1-A8myi4aKw/ids?start=0&count=20&queue=420
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4295935998
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4295791471
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4295696463
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4294785946
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4294355491
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4294332940
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4294316185
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4294322100
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4293351593
Making call: https://americas.api.riotgames.com/lol/matc

C:\Users\Tigran\AppData\Local\Temp\ipykernel_22868\989418763.py:208: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  stat_import['Bot_'+column] = new_df[column]


Making call: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/hJ1kep35r7Xg3TU52gnVkIeq8gv4AkGC7MyQQ8X67rwco-CDS4OIWi7dewHzqUTjcju-MjomWuHRlg/ids?start=0&count=20&queue=420
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4295905456
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4295848450
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4293631582
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4293595991
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4293306657
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4291613237
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4291423516
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4290885138
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4290593831
Making call: https://americas.api.riotgames.com/lol/matc

C:\Users\Tigran\AppData\Local\Temp\ipykernel_22868\989418763.py:236: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  stat_import['Sup_'+column] = new_df[column]
